# **1. Import Libraries**

In [ ]:
# Reloading modules automatically when they change
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from forecaster.data import spot
from forecaster.models import models, trainer
from forecaster.plotting.interactive import plot_predictions
from forecaster.plotting.static import plot_time_coefficients, plot_year_over_year_coefficients, plot_custom_metrics
from forecaster.data.fingrid import EXTERNAL_FEATURES

# **2. Load Data**

In [ ]:
# Load the data
file_path = '../data/spot_prices_fi_2016_2023.csv'
spot_data = spot.load_spot_data(file_path, date_col="date", price_col="elspot-fi")
plot_predictions(spot_data, slider=True)

# **3. Model Training**
Fit a linear model with LEAR features highly inspired by the following paper: [LEAR Paper](https://www.sciencedirect.com/science/article/pii/S0306261921004529?via%3Dihub)

In [ ]:
# Define different feature configurations for the model training
model_kwargs = {
    "Time + Price Lags": {
        "daily_price_lags": [1, 2, 3, 7],
        "time_features": True,
    },
    "Time + Price Lags + External Features": {
        "daily_price_lags": [1, 2, 3, 7],
        "time_features": True,
        "external_features": EXTERNAL_FEATURES,
    },
    "Time + Price Lags + External Features + External Lags": {
        "daily_price_lags": [1, 2, 3, 7],
        "time_features": True,
        "external_features": EXTERNAL_FEATURES,
        "daily_external_lags": [1, 7],
    },
}

In [ ]:
# Train the models and save predictions and coefficients
lm_predictions = {}
lm_coeffs = {}
for title, kwargs in model_kwargs.items():
    model = models.LinearModel(**kwargs)
    data = model.preprocess_data(spot_data)
    print(f"Features: {title}")
    predictions, _, coeffs = trainer.year_on_year_training(data, model)
    print("-" * 80)
    lm_predictions[title] = predictions
    lm_coeffs[title] = coeffs

# **4. Visualizations**

In [ ]:
# Visualize coefficients for Time + Price Lags + External Features + External Lags for 2023
lr_coeffs_2023 = lm_coeffs["Time + Price Lags + External Features + External Lags"][2023]

plot_time_coefficients(lr_coeffs_2023, title="Coefficients for Time Features for 2023", save_dir="../images")


In [ ]:
# Plot y_lag features for Time + Price Lags + External Features + External Lags for 2023
lr_coefficients = lm_coeffs["Time + Price Lags + External Features + External Lags"]
plot_year_over_year_coefficients(lr_coefficients, keyword="y_lag", years=[2023], save_dir="../images")

In [ ]:
# Plot custom metrics for all models
plot_custom_metrics(spot_data, lm_predictions, top_k=3, save_dir="../images")